In [1]:
import os
import sys
from pathlib import Path

# Append resources dir to path
p = os.path.dirname(os.getcwd())+'/resources/'
sys.path.append(p)

from utils import *


In [2]:
# # save mn5 version of config
# save_mn5_config()

In [8]:
# get all analysis/ and processing/ subdirs and link them in the corresponding 
# READMEs if not already done
to_update = []

# processing
d = 'processing'
with open(f'../{d}/README.md', 'r') as ifile:
    
    if d == 'processing':
        perm_dirs = ['rules', 'template_snakemake']
    else: perm_dirs = []
    
    # loop through each valid subdir
    for sub_d in Path(f'../{d}/').glob('*/'):
        stem_sub_d = sub_d.stem
        if stem_sub_d in perm_dirs: continue
        if not sub_d.is_dir(): continue
        
        fmt_sub_d = f'[{stem_sub_d}]'
        if any(fmt_sub_d in line for line in open(f'../{d}/README.md')): continue
        else: print(f'REE cant find {stem_sub_d} in readme')



REE cant find map in readme


In [29]:
str(proc_dir)

'../processing/README.md'

'template_snakemake'